In [ ]:
import datetime as dt
import pandas as pd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

df_ = pd.read_csv("/kaggle/input/omnichannel-dataset/flo_data_20k.csv")
df = df_.copy()
df.head(10)

In [ ]:
# column labels
df.columns

In [ ]:
# descriptive statistics
df.describe().T

In [ ]:
# missing values
df.isnull().sum()

In [ ]:
# data types
df.dtypes

In [ ]:
# omnichannel customers shop from both online and offline platforms. Create new variables for each customer's total number of purchases and total spending
df = df.assign(
    order_num_total=df['order_num_total_ever_online'] + df['order_num_total_ever_offline'],
    customer_value_total=df['customer_value_total_ever_online'] + df['customer_value_total_ever_offline'])
df.loc[:, ['order_num_total', 'customer_value_total']].head()

In [ ]:
# convert date variables to datetime type
date_columns = df.columns[df.columns.str.contains("date")]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.dtypes

In [ ]:
# the number of customers, the total number of products purchased, and the total spending across different shopping channels
df.groupby("order_channel").agg({"master_id": "count",
                                 "order_num_total": "sum",
                                 "customer_value_total": "sum"})

In [ ]:
# top 10 customers who generate the highest revenue
df.sort_values(by="customer_value_total", ascending=False).head(10)

In [ ]:
# top 10 customers who placed the most orders
df.sort_values(by="order_num_total", ascending=False).head(10)

In [ ]:
# function to handle the data preprocessing steps
def create_rfm(dataframe):
    dataframe = dataframe.assign(
        order_num_total=dataframe['order_num_total_ever_online'] + dataframe['order_num_total_ever_offline'],
        customer_value_total=dataframe['customer_value_total_ever_online'] + dataframe[
            'customer_value_total_ever_offline'])
    dataframe.loc[:, dataframe.columns.str.contains("date")] = dataframe.loc[:,
                                                               dataframe.columns.str.contains("date")].apply(
        pd.to_datetime)
    dataframe.groupby("order_channel").agg({"master_id": "count",
                                            "order_num_total": "sum",
                                            "customer_value_total": "sum"})
    return df

df.head()

In [ ]:
# recency, frequency, monetary metrics
today_date = dt.datetime(2021, 6, 1)
rfm = df.groupby("master_id").agg({"last_order_date": lambda date: (today_date - date.max()).days,
                                   "order_num_total": "sum",
                                   "customer_value_total": "sum"})
rfm.columns = ["recency", "frequency", "monetary"]
rfm.head()

In [ ]:
# RFM scores
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])

rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) +
                   rfm["frequency_score"].astype(str))
rfm.head()

In [ ]:
# RFM segmentation
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_risk',
    r'[1-2]5': 'cant_lose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

In [ ]:
# introducing a new women's shoe brand with prices above the general customer preferences
# to promote and sell these products, the company wants to target loyal customers (champions, loyal_customers) 
# who have previously purchased women's products. Save the IDs of these customers to a CSV file.

target_segments_new = rfm.loc[(rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")]

target_customer_ids_new = df[(df["master_id"].isin(target_segments_new.index)) &
                             (df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]

target_customer_ids_new.to_csv("target_customer_ids_new.csv", index=False)

In [ ]:
# planning a 40% discount on men's and children's products.
# the target audience includes past good customers who have not shopped for a long time (can't lose them)
# customers who are hibernating, and new customers. S
# save the IDs of these customers who are interested in men's and children's products to a CSV file.
target_segments_discount = rfm[rfm["segment"].isin(["cant_lose", "hibernating", "new_customers"])].index
target_customer_ids_discount = df[(df["master_id"].isin(target_segments_discount)) & (
        (df["interested_in_categories_12"].str.contains("ERKEK"))
        | (df["interested_in_categories_12"].str.contains("COCUK")))]["master_id"]
target_customer_ids_discount.to_csv("target_customer_ids_discount.csv", index=False)